In [9]:
try:
    import Image
except ImportError:
    from PIL import Image
import pytesseract
import requests

OAUTH_TOKEN = os.environ['OAUTH_TOKEN']
ADD_REMINDERS_ENDPOINT = 'https://slack.com/api/reminders.add'

In [3]:
def run_ocr(img_path, rotate=None):
    img=Image.open(img_path)
    
    if rotate:
        img = img.rotate(rotate)

    ocr_text = pytesseract.image_to_string(img, lang='eng')
    return ocr_text

In [4]:
def is_date(text):
    tokens = text.split() + [None]
    return tokens[0] in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

def text_to_menu_items(text):
    text_lines = text.split('\n')

    menu_items_indices = []
    for index, line in enumerate(text_lines):
        if is_date(line):
            menu_items_indices.append(index)
    
    menu_items = []
    for index in range(len(menu_items_indices)-1):
        start_index, end_index = menu_items_indices[index:index+2]
        date_component = text_lines[start_index]
        food_component = " ".join(text_lines[start_index+1:end_index])
        menu_items.append((date_component, food_component))
    return menu_items

In [10]:
def add_reminder(date, text):
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    params = {
        'time': f"on {date}",
        'token': OAUTH_TOKEN,
        'text': text,
    }
    response = requests.post(ADD_REMINDERS_ENDPOINT, headers=headers, params=params)
    print(response.text)

In [15]:
img_path = '../../data/october_food.png'
ocr_text = run_ocr(img_path)
date, food = text_to_menu_items(ocr_text)[1]
add_reminder(date, food)

{"ok":true,"reminder":{"id":"RmEY1QSZ3L","creator":"U8R740B99","user":"U8R740B99","text":"street Pop Up Black History Month a","recurring":false,"time":1570003200,"complete_ts":0}}
